<div class="alert alert-block alert-warning">
    Za probu sam koristio grafove sa<b> 
<a href="http://snap.stanford.edu/data/index.html">SNAP Stanford</a>
    </b>a posebno <b><a href="http://snap.stanford.edu/data/as-Skitter.html">Internet topology graph </a></b>. Grafovi su dati u txt i csv formatima kao <b><i>"list of edges"</i></b>. Struktura koja predstavlja graf je <b><i>dictionary</i></b>, gde je <i>key</i> broj noda, a <i>value</i> lista nodova koji su povezani sa njim. Kako to izgleda mozete videti kad pokrenete program koji sam vam poslao. Posto su nodovi predstavljeni brojevima od 0 do n-1, potrebna je i tabela sa dve kolone kako bi se svakom nodu (tj imenu noda) pridruzio broj. </div>

<h3> Prvo sam probao u Python-u, ali je Integer objekat velicine 28 bajtova </h3>

In [1]:
#import modula koji omogucava kompajliranje koda
import cython

In [2]:
%load_ext Cython

In [3]:
%%cython

def graf(file):
    Graf = {}
    f = open(file)
    for line in f:
        brojevi = line.split('\t')
        key=int(brojevi[0])
        value=int(brojevi[1][:-1])
        Graf.setdefault(key,[]).append(value)
        Graf.setdefault(value,[]).append(key)
    f.close()
    return Graf

<h3> U C-u je graf predstavljen nizom pointera na linked list strukture, ali je to bilo sporije od Python-a </h3>

<div>
<pre>
<code>
#include &lt;stdio.h&gt;
#include &lt;stdlib.h&gt;
 
typedef struct Vertices{
			int V;
			struct Vertices *Link;
			} Vertex;

/* Unapred znamo broj nodova pa mozemo da deklarisemo niz */
struct Vertices *arrayOfVertices[1000000];

void InsertNewNode(int V, Vertex **L) {
	Vertex *N, *P;
	N = (Vertex*)malloc(sizeof(Vertex));
	N->V = V;
	N->Link = NULL;
	if (*L == NULL) {
		*L = N;
	}
	else {
		P = *L;
		while (P->Link != NULL) P = P->Link;
		P->Link = N;
	}

}
struct Vertices *InsertVertexInArray(int nod_od) {

	struct Vertices *head, *temp;
	head = NULL;	/*Prazna lista */
	head = malloc(sizeof(struct Vertices));
	head->V = nod_od;
	head->Link = NULL;
	return head;
}
int main()
{
	int nod_od, nod_ka;
	FILE *fp;
	fp = fopen("proba.txt", "r");

	Vertex *L;
	L = NULL;

	while (!feof(fp))
	{
		fscanf(fp, "%d %d", &nod_od, &nod_ka);
		if (arrayOfVertices[nod_od]==NULL) {
			arrayOfVertices[nod_od] = InsertVertexInArray(nod_od);
			InsertNewNode(nod_ka, &arrayOfVertices[nod_od]);
			}
		else {

			InsertNewNode(nod_ka, &arrayOfVertices[nod_od]);
		}
		if (arrayOfVertices[nod_ka] == NULL) {
			arrayOfVertices[nod_ka] = InsertVertexInArray(nod_ka);
			InsertNewNode(nod_od, &arrayOfVertices[nod_ka]);
		}
		else {

			InsertNewNode(nod_od, &arrayOfVertices[nod_ka]);
		}	
		
	}

	fclose(fp);
	return 0;
}
</code>
</pre>
 </div>

<h3>I na kraju varijanta u C#, isto dictonary kao u Python-u</h3>

<div>
<pre>
<code>
using System;
using System.Collections.Generic;
using System.Collections.Concurrent;


namespace graphConcurrent
{
    class Program
    {
        static void Main(string[] args)
        {
          int numProcs = Environment.ProcessorCount;
          int concurrencyLevel = numProcs;
          int initialCapacity = 3000017;

          ConcurrentDictionary &lt;int, List&lt;int&gt;&gt; Graph =
                   new ConcurrentDictionary&lt;int, List&lt;int&gt;&gt;(concurrencyLevel, initialCapacity);
          string line;


          System.IO.StreamReader file =
              new System.IO.StreamReader(args[0]);

            

          while ((line = file.ReadLine()) != null)
            {
               string[] items = line.Split('\t');
               int odNoda = int.Parse(items[0]);
               int kaNodu = int.Parse(items[1]);

                
        Graph.AddOrUpdate(odNoda, new List<int> { kaNodu }, (k, v) => {Graph[odNoda].Add(kaNodu); return v;});
        Graph.AddOrUpdate(kaNodu, new List<int> { odNoda }, (v, k) => {Graph[kaNodu].Add(odNoda); return k;});
           }

            file.Close();
     
        }

        
    }
}
</code>
</pre>
</div>

Za graf od 1.7 milona nodova i 11 miliona grana (internet topology graph sa pocetka), potrebno mu je (C#) ~15 sekundi da ga generise i zauzima ~ 150 MB. Vreme potrebno za generisanje i velicina objekta rastu linearno (sa brojem grana)  pa je za generisanje npr grafa sa 100 miliona nodova potrebno ~ 150 sekundi i zauzima ~ 1.5 GB